# COMP 432 - BIXI

In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile

# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load BIXI data

In [2]:
zf = ZipFile('./data/bixi/biximontrealrentals2019-33ea73.zip')
bixi_2019_04 = pd.read_csv(zf.open('OD_2019-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_05 = pd.read_csv(zf.open('OD_2019-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_06 = pd.read_csv(zf.open('OD_2019-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_07 = pd.read_csv(zf.open('OD_2019-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_08 = pd.read_csv(zf.open('OD_2019-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_09 = pd.read_csv(zf.open('OD_2019-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_10 = pd.read_csv(zf.open('OD_2019-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_stations = pd.read_csv(zf.open('Stations_2019.csv'), dtype={'name': 'string'})

/Users/simon/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
bixi_2019_04.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0


In [4]:
bixi_2019_04.tail()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
237537,2019-04-30 23:59:27,6215,2019-05-01 00:01:57,6181,150,1
237538,2019-04-30 23:59:37,6173,2019-05-01 00:07:17,6176,459,1
237539,2019-04-30 23:59:38,6080,2019-05-01 00:10:30,6719,652,1
237540,2019-04-30 23:59:40,6226,2019-05-01 00:25:19,6175,1538,1
237541,2019-04-30 23:59:45,6065,2019-05-01 00:01:05,6063,79,1


In [5]:
bixi_2019_stations

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478,-73.570
1,4000,Jeanne-d'Arc / Ontario,45.550,-73.542
2,4001,Graham / Brookfield,45.520,-73.630
3,4002,Graham / Wicksteed,45.517,-73.640
4,5002,St-Charles / Montarville,45.534,-73.515
...,...,...,...,...
614,7149,16e avenue / Jean-Talon,45.562,-73.595
615,7150,du Souvenir / Chomedey,45.552,-73.744
616,7151,Métro Montmorency (de l'Avenir / Jacques-Tétre...,45.558,-73.720
617,7152,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692


## Load weather data

In [6]:
# Note: Use Date/Time column instead of Time column
weather_2019_04 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_04-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_05 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_05-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_06 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_06-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_07 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_07-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_08 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_08-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_09 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_09-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_10 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_10-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})

In [7]:
weather_2019_04

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,<NA>
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,<NA>
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-30 19:00:00,2019,4,30,19:00,11.400,...,nan,48.300,nan,102.370,nan,nan,nan,nan,nan,Mostly Cloudy
716,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-30 20:00:00,2019,4,30,20:00,9.800,...,nan,24.100,nan,102.480,nan,nan,nan,nan,nan,<NA>
717,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-30 21:00:00,2019,4,30,21:00,8.900,...,nan,24.100,nan,102.500,nan,nan,nan,nan,nan,<NA>
718,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-30 22:00:00,2019,4,30,22:00,8.200,...,nan,24.100,nan,102.520,nan,nan,nan,nan,nan,Cloudy


## Data Cleaning

### Checking if there are missing data
Only the weather data is missing some values

In [8]:
bixi_2019_04.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [9]:
bixi_2019_05.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [10]:
bixi_2019_06.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [11]:
bixi_2019_07.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [12]:
bixi_2019_08.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [13]:
bixi_2019_09.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [14]:
 bixi_2019_10.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
dtype: int64

In [15]:
bixi_2019_stations.isnull().sum()

Code         0
name         0
latitude     0
longitude    0
dtype: int64

In [16]:
weather_2019_04.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              720
Dew Point Temp (°C)      0
Dew Point Temp Flag    720
Rel Hum (%)              0
Rel Hum Flag           720
Wind Dir (10s deg)       1
Wind Dir Flag          720
Wind Spd (km/h)          0
Wind Spd Flag          720
Visibility (km)          0
Visibility Flag        720
Stn Press (kPa)          0
Stn Press Flag         720
Hmdx                   720
Hmdx Flag              720
Wind Chill             608
Wind Chill Flag        720
Weather                368
dtype: int64

In [17]:
weather_2019_05.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              744
Dew Point Temp (°C)      0
Dew Point Temp Flag    744
Rel Hum (%)              0
Rel Hum Flag           744
Wind Dir (10s deg)       1
Wind Dir Flag          744
Wind Spd (km/h)          0
Wind Spd Flag          744
Visibility (km)          0
Visibility Flag        744
Stn Press (kPa)          0
Stn Press Flag         744
Hmdx                   733
Hmdx Flag              744
Wind Chill             744
Wind Chill Flag        744
Weather                412
dtype: int64

In [18]:
weather_2019_06.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              720
Dew Point Temp (°C)      0
Dew Point Temp Flag    720
Rel Hum (%)              0
Rel Hum Flag           720
Wind Dir (10s deg)       2
Wind Dir Flag          720
Wind Spd (km/h)          0
Wind Spd Flag          720
Visibility (km)          0
Visibility Flag        720
Stn Press (kPa)          0
Stn Press Flag         720
Hmdx                   554
Hmdx Flag              720
Wind Chill             720
Wind Chill Flag        720
Weather                436
dtype: int64

In [19]:
weather_2019_07.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                1
Temp Flag              744
Dew Point Temp (°C)      1
Dew Point Temp Flag    744
Rel Hum (%)              1
Rel Hum Flag           744
Wind Dir (10s deg)       1
Wind Dir Flag          744
Wind Spd (km/h)          1
Wind Spd Flag          744
Visibility (km)          1
Visibility Flag        744
Stn Press (kPa)          1
Stn Press Flag         744
Hmdx                   207
Hmdx Flag              744
Wind Chill             744
Wind Chill Flag        744
Weather                483
dtype: int64

In [20]:
weather_2019_08.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              744
Dew Point Temp (°C)      0
Dew Point Temp Flag    744
Rel Hum (%)              0
Rel Hum Flag           744
Wind Dir (10s deg)       0
Wind Dir Flag          744
Wind Spd (km/h)          0
Wind Spd Flag          744
Visibility (km)          0
Visibility Flag        744
Stn Press (kPa)          0
Stn Press Flag         744
Hmdx                   387
Hmdx Flag              744
Wind Chill             744
Wind Chill Flag        744
Weather                471
dtype: int64

In [21]:
weather_2019_09.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              720
Dew Point Temp (°C)      0
Dew Point Temp Flag    720
Rel Hum (%)              0
Rel Hum Flag           720
Wind Dir (10s deg)       0
Wind Dir Flag          720
Wind Spd (km/h)          0
Wind Spd Flag          720
Visibility (km)          0
Visibility Flag        720
Stn Press (kPa)          0
Stn Press Flag         720
Hmdx                   643
Hmdx Flag              720
Wind Chill             720
Wind Chill Flag        720
Weather                419
dtype: int64

In [22]:
weather_2019_10.isnull().sum()

Longitude (x)            0
Latitude (y)             0
Station Name             0
Climate ID               0
Date/Time                0
Year                     0
Month                    0
Day                      0
Time                     0
Temp (°C)                0
Temp Flag              744
Dew Point Temp (°C)      0
Dew Point Temp Flag    744
Rel Hum (%)              0
Rel Hum Flag           744
Wind Dir (10s deg)       0
Wind Dir Flag          744
Wind Spd (km/h)          0
Wind Spd Flag          744
Visibility (km)          0
Visibility Flag        744
Stn Press (kPa)          0
Stn Press Flag         744
Hmdx                   744
Hmdx Flag              744
Wind Chill             744
Wind Chill Flag        744
Weather                402
dtype: int64

### Checking the contents of the "Weather" column
Some of the weather values contains more than 1 condition. We'll have to take this into account while training.

In [23]:
weather_2019_04['Weather'].value_counts()

Rain                       89
Cloudy                     63
Mostly Cloudy              52
Clear                      35
Mainly Clear               33
Rain,Fog                   23
Snow                       17
Drizzle,Fog                 9
Drizzle                     4
Freezing Rain               4
Freezing Rain,Fog           3
Freezing Drizzle,Fog        3
Snow Showers                3
Moderate Rain,Fog           2
Snow,Fog                    2
Freezing Drizzle,Snow       2
Drizzle,Snow,Fog            2
Rain,Snow                   2
Moderate Snow               1
Rain Showers                1
Drizzle,Snow Grains,Fog     1
Fog                         1
Name: Weather, dtype: Int64

In [24]:
weather_2019_05['Weather'].value_counts()

Cloudy               75
Mostly Cloudy        74
Rain                 73
Clear                26
Mainly Clear         25
Rain,Fog             18
Drizzle              13
Rain Showers         13
Fog                   7
Drizzle,Fog           4
Moderate Rain         2
Rain,Rain Showers     1
Rain Showers,Fog      1
Name: Weather, dtype: Int64

In [25]:
weather_2019_06['Weather'].value_counts()

Mostly Cloudy                 85
Mainly Clear                  56
Cloudy                        41
Clear                         39
Rain                          33
Rain Showers                  18
Rain,Fog                       5
Thunderstorms,Rain Showers     3
Drizzle                        2
Thunderstorms                  1
Thunderstorms,Rain             1
Name: Weather, dtype: Int64

In [26]:
weather_2019_07['Weather'].value_counts()

Mostly Cloudy                 88
Mainly Clear                  68
Clear                         57
Cloudy                        28
Rain Showers                  15
Thunderstorms,Rain Showers     4
Drizzle                        1
Name: Weather, dtype: Int64

In [27]:
weather_2019_08['Weather'].value_counts()

Mostly Cloudy                 75
Clear                         71
Mainly Clear                  61
Cloudy                        27
Rain                          19
Rain Showers                  14
Rain,Fog                       2
Drizzle                        1
Moderate Rain Showers          1
Thunderstorms,Rain Showers     1
Fog                            1
Name: Weather, dtype: Int64

In [28]:
weather_2019_09['Weather'].value_counts()

Mostly Cloudy                          70
Clear                                  53
Mainly Clear                           47
Cloudy                                 45
Rain                                   43
Rain Showers                           18
Fog                                     9
Rain,Fog                                8
Drizzle                                 2
Thunderstorms,Rain Showers              2
Drizzle,Fog                             2
Moderate Rain Showers                   1
Thunderstorms,Moderate Rain Showers     1
Name: Weather, dtype: Int64

In [29]:
weather_2019_10['Weather'].value_counts()

Mostly Cloudy                              76
Rain                                       53
Clear                                      53
Cloudy                                     47
Rain,Fog                                   27
Mainly Clear                               27
Drizzle,Fog                                17
Moderate Rain,Fog                          14
Fog                                        11
Rain Showers                                8
Moderate Rain                               4
Drizzle                                     2
Thunderstorms,Moderate Rain Showers         1
Thunderstorms,Heavy Rain Showers            1
Thunderstorms,Moderate Rain Showers,Fog     1
Name: Weather, dtype: Int64

### Checking data stats
To see if there are any weird anomalies or outliers

In [30]:
bixi_2019_04.describe()

,start_station_code,end_station_code,duration_sec,is_member
count,237542.000,237542.000,237542.000,237542.000
mean,6372.634,6372.351,772.981,0.854
std,439.040,454.168,633.223,0.353
min,4000.000,4000.000,61.000,0.000
25%,6131.000,6112.000,334.000,1.000
50%,6219.000,6217.000,593.000,1.000
75%,6427.000,6434.000,1036.000,1.000
max,10002.000,10002.000,7197.000,1.000


In [31]:
bixi_2019_stations.describe()

,Code,latitude,longitude
count,619.000,619.000,619.000
mean,6483.165,45.520,-73.586
std,475.126,0.034,0.036
min,4000.000,45.415,-73.744
25%,6160.500,45.501,-73.606
50%,6350.000,45.523,-73.578
75%,6929.500,45.540,-73.565
max,10002.000,45.651,-73.490


In [32]:
weather_2019_04.describe()

,Longitude (x),Latitude (y),Climate ID,Year,Month,Day,Temp (°C),Temp Flag,Dew Point Temp (°C),Dew Point Temp Flag,...,Wind Spd (km/h),Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag
count,720.000,720.000,720.000,720.000,720.000,720.000,720.000,0.000,720.000,0.000,...,720.000,0.000,720.000,0.000,720.000,0.000,0.000,0.000,112.000,0.000
mean,-73.740,45.470,7025251.000,2019.000,4.000,15.500,5.356,nan,-1.258,nan,...,19.478,nan,28.942,nan,101.036,nan,nan,nan,-7.304,nan
std,0.000,0.000,0.000,0.000,0.000,8.661,4.823,nan,5.850,nan,...,10.534,nan,13.400,nan,0.941,nan,nan,nan,2.134,nan
min,-73.740,45.470,7025251.000,2019.000,4.000,1.000,-7.000,nan,-16.000,nan,...,0.000,nan,0.800,nan,98.500,nan,nan,nan,-14.000,nan
25%,-73.740,45.470,7025251.000,2019.000,4.000,8.000,2.075,nan,-5.200,nan,...,11.000,nan,24.100,nan,100.430,nan,nan,nan,-8.000,nan
50%,-73.740,45.470,7025251.000,2019.000,4.000,15.500,5.400,nan,-1.700,nan,...,18.000,nan,24.100,nan,101.080,nan,nan,nan,-7.000,nan
75%,-73.740,45.470,7025251.000,2019.000,4.000,23.000,8.300,nan,3.500,nan,...,27.000,nan,48.300,nan,101.750,nan,nan,nan,-6.000,nan
max,-73.740,45.470,7025251.000,2019.000,4.000,30.000,18.300,nan,11.500,nan,...,50.000,nan,48.300,nan,102.920,nan,nan,nan,-2.000,nan


## Merging data

### Merging all 2019 trips

In [33]:
bixi_2019_all = pd.concat([bixi_2019_04, bixi_2019_05, bixi_2019_06, bixi_2019_07, bixi_2019_08, bixi_2019_09, bixi_2019_10])
bixi_2019_all

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0
...,...,...,...,...,...,...
617190,2019-10-31 23:52:45,6901,2019-11-01 00:05:15,6063,749,1
617191,2019-10-31 23:53:59,6218,2019-11-01 00:23:19,6269,1760,1
617192,2019-10-31 23:55:26,7149,2019-11-01 00:11:33,6338,966,1
617193,2019-10-31 23:56:28,6199,2019-11-01 00:09:13,6003,764,1


#### Adding start and end station data
May take a while to run

In [34]:
bixi_2019_all_s = bixi_2019_all.merge(bixi_2019_stations, left_index=True, left_on='start_station_code', right_on='Code').drop('Code', 1).rename({'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, axis=1).merge(bixi_2019_stations, left_index=True, left_on='end_station_code', right_on='Code').drop('Code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)

In [35]:
bixi_2019_all_s

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude
129,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1,Métro Champ-de-Mars (Viger / Sanguinet),45.510,-73.557,Larivière / de Lorimier,45.529,-73.559
129,2019-04-16 16:36:49,6001,2019-04-16 16:51:49,6132,899,1,Métro Champ-de-Mars (Viger / Sanguinet),45.510,-73.557,Larivière / de Lorimier,45.529,-73.559
129,2019-04-17 23:57:03,6001,2019-04-18 00:07:34,6132,631,1,Métro Champ-de-Mars (Viger / Sanguinet),45.510,-73.557,Larivière / de Lorimier,45.529,-73.559
129,2019-04-19 00:15:05,6001,2019-04-19 00:26:36,6132,691,1,Métro Champ-de-Mars (Viger / Sanguinet),45.510,-73.557,Larivière / de Lorimier,45.529,-73.559
129,2019-04-23 23:55:37,6001,2019-04-24 00:07:27,6132,709,1,Métro Champ-de-Mars (Viger / Sanguinet),45.510,-73.557,Larivière / de Lorimier,45.529,-73.559
...,...,...,...,...,...,...,...,...,...,...,...,...
617,2019-10-21 19:52:12,7152,2019-10-21 20:36:58,7152,2686,1,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692
617,2019-10-24 14:29:52,7152,2019-10-24 15:07:36,7152,2263,1,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692
617,2019-10-24 14:30:26,7152,2019-10-24 15:07:33,7152,2227,1,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692
617,2019-10-26 13:48:15,7152,2019-10-26 14:27:33,7152,2358,0,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692,Parc St-Claude (7e avenue / 8e rue),45.563,-73.692


Confirming that all trips have a lon/lat associated with it

In [36]:
bixi_2019_all_s.isnull().sum()

start_date            0
start_station_code    0
end_date              0
end_station_code      0
duration_sec          0
is_member             0
start_name            0
start_latitude        0
start_longitude       0
end_name              0
end_latitude          0
end_longitude         0
dtype: int64

### Merging weather data

In [37]:
weather_2019_all = pd.concat([weather_2019_04, weather_2019_05, weather_2019_06, weather_2019_07, weather_2019_08, weather_2019_09, weather_2019_10])
weather_2019_all

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,<NA>
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,<NA>
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-31 19:00:00,2019,10,31,19:00,11.900,...,nan,4.000,nan,99.110,nan,nan,nan,nan,nan,"Moderate Rain,Fog"
740,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-31 20:00:00,2019,10,31,20:00,10.800,...,nan,8.100,nan,98.970,nan,nan,nan,nan,nan,"Rain,Fog"
741,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-31 21:00:00,2019,10,31,21:00,9.900,...,nan,12.900,nan,99.020,nan,nan,nan,nan,nan,Moderate Rain
742,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-31 22:00:00,2019,10,31,22:00,9.000,...,nan,24.100,nan,98.850,nan,nan,nan,nan,nan,Rain


### Merging weather with trips

In [38]:
bixi_2019_all_sw = bixi_2019_all_s.merge(weather_2019_all[['Date/Time', 'Weather', 'Temp (°C)']], left_on=bixi_2019_all_s['start_date'].dt.floor('H'), right_on='Date/Time').drop('Date/Time', 1).rename({'Weather': 'start_weather', 'Temp (°C)': 'start_temp'}, axis=1)

In [39]:
bixi_2019_all_sw.sample(10)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude,start_weather,start_temp
4656511,2019-09-25 21:08:35,6107,2019-09-25 21:18:41,6070,606,1,St-Mathieu / Ste-Catherine,45.494,-73.579,Milton / University,45.506,-73.576,<NA>,16.500
5343202,2019-09-07 02:00:53,6078,2019-09-07 02:13:05,6217,732,1,St-Alexandre / Ste-Catherine,45.505,-73.568,Vallières / St-Laurent,45.519,-73.584,<NA>,14.900
4881365,2019-05-12 10:06:08,6268,2019-05-12 10:19:21,6022,793,1,Chambord / Beaubien,45.540,-73.599,William-Tremblay / Molson,45.542,-73.565,Mostly Cloudy,12.900
1560891,2019-07-27 21:30:44,6149,2019-07-27 21:41:18,7077,634,1,Chapleau / du Mont-Royal,45.539,-73.569,Jean Langlois / Fullum,45.526,-73.546,<NA>,24.900
3914204,2019-05-22 08:02:38,7020,2019-05-22 08:27:04,6753,1465,1,St-Germain / Hochelaga,45.544,-73.554,Centre ÉPIC (St-Zotique / 40e avenue),45.574,-73.575,<NA>,13.500
4457914,2019-06-12 14:31:37,6027,2019-06-12 14:42:49,6078,671,1,de Maisonneuve / Mansfield (sud),45.502,-73.573,St-Alexandre / Ste-Catherine,45.505,-73.568,<NA>,23.300
4730075,2019-06-13 10:43:40,6070,2019-06-13 10:46:11,6027,150,1,Milton / University,45.506,-73.576,de Maisonneuve / Mansfield (sud),45.502,-73.573,Cloudy,18.300
334559,2019-08-08 15:21:45,6380,2019-08-08 15:25:26,6374,220,1,Parc J.-Arthur-Champagne (de Chambly / du Mont...,45.552,-73.562,15e avenue / Masson,45.554,-73.571,<NA>,28.400
1566136,2019-06-17 17:51:25,7031,2019-06-17 18:00:19,6222,534,1,Berri / Rachel,45.523,-73.577,de l'Esplanade / du Mont-Royal,45.518,-73.588,<NA>,23.100
524671,2019-06-25 18:01:47,7014,2019-06-25 18:19:16,6903,1048,0,Métro Université de Montréal (Édouard-Montpeti...,45.504,-73.618,St-Dominique / Napoléon,45.517,-73.577,<NA>,24.800


In [40]:
bixi_2019_all_sw.isnull().sum()

start_date                  0
start_station_code          0
end_date                    0
end_station_code            0
duration_sec                0
is_member                   0
start_name                  0
start_latitude              0
start_longitude             0
end_name                    0
end_latitude                0
end_longitude               0
start_weather         3459073
start_temp               1494
dtype: int64

In [41]:
bixi_2019_all_sw.describe()

,duration_sec,is_member,start_latitude,start_longitude,end_latitude,end_longitude,start_temp
count,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5442288.000,5440794.000
mean,811.042,0.820,45.517,-73.580,45.516,-73.578,18.999
std,625.547,0.384,0.023,0.023,0.024,0.023,6.344
min,61.000,0.000,45.415,-73.744,45.415,-73.744,-2.100
25%,369.000,1.000,45.503,-73.592,45.502,-73.589,14.400
50%,647.000,1.000,45.520,-73.576,45.519,-73.575,19.300
75%,1090.000,1.000,45.532,-73.565,45.532,-73.563,24.000
max,7199.000,1.000,45.651,-73.490,45.651,-73.490,32.800
